In [1]:
import os
os.chdir('/data/hyeryung/mucoco')

from transformers import AutoModelForMaskedLM, AutoModelForSequenceClassification, AutoTokenizer
from new_module.locate.new_locate_utils import LocateMachine
import pandas as pd

In [2]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer

In [3]:
def unravel(outputs_df):
    outputs_df=outputs_df.explode('generations',ignore_index=True)
    outputs_df['prompt']=outputs_df['prompt'].apply(lambda x: x['text'])
    outputs_df['generations']=outputs_df['generations'].apply(lambda x: x['text'] if isinstance(x, dict) else x)
    outputs_df = outputs_df.dropna().reset_index(drop=True)
    return outputs_df
toxic_gen=pd.read_json('new_module/data/toxicity-avoidance/dev_set.jsonl',lines=True)
toxic_gen=unravel(toxic_gen)
prompt = '“I went to use the bathroom, and I went in and Lance was bent'
prediction = toxic_gen.loc[toxic_gen['prompt']==prompt,'generations'].tolist()
ckpt_path = '/data/hyeryung/loc_edit/models/roberta-base-jigsaw-toxicity-classifier-energy-training/step_1000_best_checkpoint/'
model = AutoModelForSequenceClassification.from_pretrained(ckpt_path)
tokenizer = AutoTokenizer.from_pretrained(ckpt_path)
device='cuda'
model = model.to(device)
loc_machine=LocateMachine(model,tokenizer)
res = loc_machine.locate_main(prediction, "attention", max_num_tokens = 6, unit="word", num_layer=10, label_id=0)

In [4]:
mlm_tokenizer = AutoTokenizer.from_pretrained('roberta-base',cache_dir='/data/hyeryung/hf_cache')
mlm = AutoModelForMaskedLM.from_pretrained('roberta-base',cache_dir='/data/hyeryung/hf_cache')
mlm = mlm.to(device)

In [5]:
masked_text = res

In [17]:
## replace tokens at the indices with mask tokens
inputs = mlm_tokenizer(
    masked_text, return_tensors="pt", padding=True, truncation=True
)
inputs = inputs.to(device) 
# inputs = mlm_tokenizer(
#     source_text + ' ' + masked_text[0], return_tensors="pt", add_special_tokens=False
# )

## make predictions for the masked indices
with torch.no_grad():
    logits = mlm(**inputs).logits


In [18]:
special_token_ids = mlm_tokenizer.convert_tokens_to_ids(mlm_tokenizer.all_special_tokens)
logits[:, :, special_token_ids] = -float("inf")

In [22]:
indices_in_mlm_tokens = (
    inputs.input_ids == mlm_tokenizer.mask_token_id
).nonzero(as_tuple=True)
# print(f"indices_in_mlm_tokens: {indices_in_mlm_tokens}")
## get top k tokens for each index

predicted_token_ids = torch.topk(
    logits[indices_in_mlm_tokens[0], indices_in_mlm_tokens[1], :],
    k=10,
    dim=-1,
)
# print(f"predicted_token_ids: {predicted_token_ids}")
# print(f"mlm_tokenizer.batch_decode(predicted_token_ids.indices): {mlm_tokenizer.batch_decode(predicted_token_ids.indices)}")


In [24]:
predicted_token_ids.values.shape

torch.Size([65, 10])

In [25]:
predicted_token_ids.indices.shape

torch.Size([65, 10])

In [26]:
## replace tokens at the indices with mask tokens
inputs = mlm_tokenizer(
    masked_text, return_tensors="pt", padding=True, truncation=True
)
inputs = inputs.to(device) 
# inputs = mlm_tokenizer(
#     source_text + ' ' + masked_text[0], return_tensors="pt", add_special_tokens=False
# )

## make predictions for the masked indices
with torch.no_grad():
    logits = mlm(**inputs).logits

special_token_ids = mlm_tokenizer.convert_tokens_to_ids(mlm_tokenizer.all_special_tokens)
logits[:, :, special_token_ids] = -float("inf")

indices_in_mlm_tokens = (
    inputs.input_ids == mlm_tokenizer.mask_token_id
).nonzero(as_tuple=True)

## get top k tokens for each index
predicted_token_ids = torch.topk(
    logits[indices_in_mlm_tokens[0], indices_in_mlm_tokens[1], :],
    k=10,
    dim=-1,
)
print(f"predicted_token_ids: {predicted_token_ids}")
print(f"mlm_tokenizer.batch_decode(predicted_token_ids.indices): {mlm_tokenizer.batch_decode(predicted_token_ids.indices)}")


predicted_token_ids: torch.return_types.topk(
values=tensor([[22.0476, 19.4729, 17.5394, 16.8642, 16.5963, 16.4827, 16.4021, 15.7711,
         15.7436, 15.3570],
        [17.5601, 16.3528, 15.0402, 14.9633, 14.3649, 13.8561, 13.2594, 13.2089,
         12.6610, 12.5642],
        [16.7789, 15.7252, 15.5652, 15.4932, 15.3546, 15.3041, 15.2817, 15.1390,
         14.8545, 14.6381],
        [16.9867, 16.6328, 16.1767, 14.9714, 14.3802, 13.4213, 13.3534, 13.3342,
         12.8598, 12.8383],
        [15.5927, 15.5903, 15.5631, 14.8100, 14.4180, 14.1002, 13.6288, 13.5641,
         13.5315, 13.0986],
        [16.8475, 16.7198, 16.0989, 15.9949, 15.4332, 15.2327, 15.2147, 14.9328,
         14.8248, 14.5735],
        [19.4643, 14.3413, 13.1266, 12.6918, 12.6008, 12.3762, 11.8818, 11.5772,
         11.5281, 11.4883],
        [17.3266, 15.7880, 15.7409, 15.6494, 15.2200, 14.8165, 14.7254, 14.6882,
         14.5433, 14.4847],
        [16.9962, 16.8003, 16.3852, 16.3049, 15.7828, 14.8172, 14.3738, 14.

# Check for discrepancy

In [1]:
#!/usr/bin/env python
# coding: utf-8

from itertools import chain
import math
import argparse
import json
import logging
import os
import time
os.chdir('/data/hyeryung/mucoco')
import numpy as np
import pandas as pd
import torch
import transformers
from transformers import AutoConfig, AutoModelForMaskedLM, AutoTokenizer

from copy import deepcopy
import new_module.losses as lossbuilder
import new_module.losses_old as lossbuilder_old
import wandb
# from new_module.decode_utils import (
#     beam_rerank_v0,
#     beam_rerank_v1,
#     beam_rerank_v2,
#     combi_rerank,
# )
from new_module.new_decode_utils import get_beam_hypotheses, get_combi_hypotheses, final_reranking
from new_module.evaluate_wandb import evaluate_main
from new_module.locate.new_locate_utils import LocateMachine
from new_module.locate.locate_utils import locate_main
from new_module.utils.robertacustom import RobertaCustomForSequenceClassification

logging.basicConfig(level=logging.DEBUG, format="%(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(os.environ.get("LOGGING_LEVEL", logging.DEBUG))
import joblib
config = joblib.load('config.pkl')

In [3]:
main_start_time = time.time()

if not config.get("model_tag", None):
    if "energy-training" in config["model_paths"][1]:
        config["model_tag"] = "em"
    else:
        config["model_tag"] = "clsf"

    if (config["task"] == "formality") and ("gyafc" in config["model_paths"][1]):
        config["model_tag"] += "-gyafc"

if config["resume"]:
    logger.info("resuming from a previous run")
    run = wandb.init(
        project=config["wandb_project"],
        entity=config["wandb_entity"],
        id=config["wandb_run_id"],
        resume="must",
    )
else:
    run = wandb.init(
        project=config["wandb_project"],
        entity=config["wandb_entity"],
        config=config,
    )

run_id = run.path.split("/")[-1]
display_name = f"{run_id}"


outdir = os.path.join(config["output_dir_prefix"], display_name)
os.makedirs(outdir, exist_ok=True)
outfile = f"{outdir}/outputs_epsilon{config['min_epsilons'][0]}.txt"
run.summary["outfile_path"] = outfile


class dummyArgs:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

build_loss_args = dummyArgs(**config["build_loss_dict"])

## load data
if (config["task"] == "toxicity") or (config["task"] == "sentiment"):
    source_dataset = [
        json.loads(l)[config["jsonl_primary_key"]][config["jsonl_secondary_key"]]
        for l in open(config["source_data"])
    ]
    generation_dataset = [
        json.loads(l)["generations"] for l in open(config["source_data"])
    ]
elif (config["task"] == "formality") or (config["task"] == "sentiment-lewis-compr"):
    with open(config["source_data"], "r") as f:
        generation_dataset = [line.rstrip('\n') for line in f.readlines()]
    source_dataset = ["" for l in generation_dataset]

# check if outfile exists
if (config["resume"]) and (os.path.exists(outfile)):

    with open(outfile, "r") as f:
        existing_gens = [x.rstrip("\n") for x in f.readlines()]
    resume_idx = len(existing_gens)
    if resume_idx == len(source_dataset):
        logger.debug("output file is already complete. skipping this run.")
        raise
    elif resume_idx < len(source_dataset):
        logger.info(
            f"output file already exists but is incomplete. resuming from index: {resume_idx}"
        )
        outf = open(outfile, "a")
        int_outf = open(outfile+".intermediate", "a")
    else:
        logger.critical(
            f"output file seems to be corrupted. The file length is {resume_idx}, where the size of source_dataset is {len(source_dataset)}"
        )
        raise
else:
    resume_idx = 0
    outf = open(outfile, "w")
    int_outf = open(outfile+".intermediate", "w")


## load tokenizer, models, define losses
name2tokenizer = {}
name2model = {}
name2config = {}
loss2tokenizer = {}
embed_luts = []

for i, model_path in enumerate(config["model_paths"]):
    if (
        model_path not in name2model
    ):  # making sure we are not loading the model twice in case some constraints use the same model.
        try:
            name2tokenizer[config["tokenizer_paths"][i]] = AutoTokenizer.from_pretrained(
                config["tokenizer_paths"][i],
                cache_dir=config["cache_dir"],
                use_fast=True,
            )
        except:
            name2tokenizer[config["tokenizer_paths"][i]] = AutoTokenizer.from_pretrained(
                config["tokenizer_paths"][i],
                cache_dir=config["cache_dir"],
                use_fast=False,
            )

        name2config[model_path] = AutoConfig.from_pretrained(
            model_path, cache_dir=config["cache_dir"]
        )

        if config["model_types"][i] == "RobertaCustomForSequenceClassification":
            name2model[model_path] = lossbuilder.ModelWrapper(
                RobertaCustomForSequenceClassification.from_pretrained(
                    model_path,
                    config=name2config[model_path],
                    cache_dir=config["cache_dir"],
                )
            )
        else:
            name2model[model_path] = lossbuilder.ModelWrapper(
                getattr(transformers, config["model_types"][i]).from_pretrained(
                    model_path,
                    config=name2config[model_path],
                    cache_dir=config["cache_dir"],
                )
            )
        name2model[model_path].eval()
        name2model[model_path].to(config['device'])

    input_embeds = name2model[model_path].get_input_embeddings()
    if isinstance(input_embeds, torch.nn.Sequential):
        input_embeds = input_embeds[0]
    embed_luts.append(input_embeds)

    if config["target_type"] == "embeds":
        embed_luts[-1].requires_grad = False

mlm_tokenizer = AutoTokenizer.from_pretrained("roberta-base")
mlm = None if config["method"] == "mlm-beamsearch-v2" else AutoModelForMaskedLM.from_pretrained("roberta-base").to(config['device'])


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Starting new HTTPS connection (1): api.wandb.ai:443
https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 1830
https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 377
wandb: Currently logged in as: hayleyson. Use `wandb login --relogin` to force relogin
Popen(['git', 'cat-file', '--batch-check'], cwd=/data/hyeryung/mucoco, stdin=<valid stream>, shell=False, universal_newlines=False)


Starting new HTTPS connection (1): huggingface.co:443
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/generation_config.json HTTP/1.1" 200 0


50265


https://huggingface.co:443 "HEAD /roberta-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /roberta-base/resolve/main/config.json HTTP/1.1" 200 0


In [4]:

lossfns = []
for i, loss in enumerate(config["losses"]):
    lossfns.append(
        lossbuilder.build_loss(
            loss,
            name2model[config["model_paths"][i]],
            name2tokenizer[config["tokenizer_paths"][i]],
            build_loss_args,
        )
    )
    lossfns[i].tokenizer.add_special_tokens({"mask_token": mlm_tokenizer.mask_token})
    loss2tokenizer[loss] = lossfns[i].tokenizer


In [5]:

loss_weights = [1 - wandb.config.closs_weight, wandb.config.closs_weight]
interrupted = False
# for text_id in range(len(source_dataset))[resume_idx:]:
text_id = 0
source_text = source_dataset[text_id]
if source_text == "":
    source_text = lossfns[0].tokenizer.bos_token

if (config["task"] == "toxicity") or (config["task"] == "sentiment"):
    AR_prediction_all = [x["text"] for x in generation_dataset[text_id]]
    # predicted_batches = [x["tokens"] for x in generation_dataset[text_id]]
    # predicted_batches = [
    #     torch.tensor([x], dtype=torch.long, device=config["device"])
    #     for x in predicted_batches
    # ]
    
elif (config["task"] == "formality") or (
    config["task"] == "sentiment-lewis-compr"
):
    AR_prediction_all = [generation_dataset[text_id]]

curr_num_samples = len(AR_prediction_all)

curr_loss = torch.zeros(len(AR_prediction_all)).to(config['device'])
logging_loss = torch.zeros((len(AR_prediction_all),2)).to(config['device'])
edit_yn = torch.ones(len(AR_prediction_all), dtype=torch.bool).to(config['device'])
        
for lossid, lossname in enumerate(config["losses"]):
    with torch.no_grad():
        lossvalue = lossfns[lossid].compute_gold_loss(
            source_text, AR_prediction_all,
            label_id=config['target_label_ids'][lossid],
        )
        torch.cuda.empty_cache()
    curr_loss += loss_weights[lossid] * lossvalue
    logging_loss[:, lossid] = lossvalue.clone()

In [6]:

# define an object to locate problematic phrases
locator = LocateMachine(lossfns[1].model, lossfns[1].tokenizer)

label_ids = config["target_label_ids"]  # target label's ids for each loss

run.summary["prep_time"] = time.time() - main_start_time
## beginning of main logic
decode_start_time = time.time()
# text_id = 0
if config["resume"]:
    num_skipped = run.summary.get("num_skipped", 0)
    num_edited = run.summary.get("num_edited", 0)
    num_decoded_tokens = run.summary.get("num_decoded_tokens", 0)
else:
    num_skipped = 0
    num_edited = 0
    num_decoded_tokens = 0


loss_weights = [1 - wandb.config.closs_weight, wandb.config.closs_weight]
interrupted = False
# for text_id in range(len(source_dataset))[resume_idx:]:
text_id = 0
source_text = source_dataset[text_id]
if source_text == "":
    source_text = lossfns[0].tokenizer.bos_token

if (config["task"] == "toxicity") or (config["task"] == "sentiment"):
    AR_prediction_all = [x["text"] for x in generation_dataset[text_id]]
    # predicted_batches = [x["tokens"] for x in generation_dataset[text_id]]
    # predicted_batches = [
    #     torch.tensor([x], dtype=torch.long, device=config["device"])
    #     for x in predicted_batches
    # ]
    
elif (config["task"] == "formality") or (
    config["task"] == "sentiment-lewis-compr"
):
    AR_prediction_all = [generation_dataset[text_id]]

curr_num_samples = len(AR_prediction_all)

       
running_text = best_text = deepcopy(AR_prediction_all)



In [7]:
masked_text = locator.locate_main(running_text, 
                        method = config['locate_method'], 
                        max_num_tokens = wandb.config.num_edit_token_per_step, 
                        unit = config['locate_unit'], 
                        num_layer = -2, #penultimate
                        label_id = config['target_label_ids'][1])

In [8]:
## replace tokens at the indices with mask tokens
                
inputs = mlm_tokenizer(
    masked_text, return_tensors="pt", padding=True, truncation=True
)
inputs = inputs.to(config['device']) 
masked_sequence=inputs['input_ids']


## make predictions for the masked indices
with torch.no_grad():
    logits = mlm(**inputs).logits

special_token_ids = mlm_tokenizer.convert_tokens_to_ids(mlm_tokenizer.all_special_tokens)
logits[:, :, special_token_ids] = -float("inf")


indices_in_mlm_tokens = (
    inputs.input_ids == mlm_tokenizer.mask_token_id
).nonzero(as_tuple=True)

## get top k tokens for each index
predicted_token_ids = torch.topk(
    logits[indices_in_mlm_tokens[0], indices_in_mlm_tokens[1], :],
    k=config['k_per_location'],
    dim=-1,
)


In [10]:
indices_in_mlm_tokens

(tensor([0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2], device='cuda:0'),
 tensor([ 7, 11, 16, 17, 18, 24,  4,  7, 10, 11, 19, 20, 32, 33, 34],
        device='cuda:0'))

In [9]:
predicted_token_ids

torch.return_types.topk(
values=tensor([[15.873, 15.323, 15.225, 14.497, 13.819, 13.768, 13.668, 13.610, 13.591,
         13.532],
        [16.084, 14.017, 13.802, 13.463, 13.243, 13.119, 12.944, 12.736, 12.726,
         12.509],
        [13.976, 13.951, 13.586, 13.123, 12.936, 12.907, 12.618, 12.576, 12.389,
         12.001],
        [13.024, 12.771, 12.487, 12.358, 12.161, 12.140, 12.063, 11.893, 11.850,
         11.711],
        [19.656, 15.377, 13.710, 12.652, 12.214, 11.788, 11.575, 11.137, 11.029,
         10.996],
        [17.550, 15.963, 15.902, 15.694, 15.647, 15.253, 14.809, 14.134, 14.113,
         14.093],
        [14.363, 13.436, 13.361, 12.893, 12.522, 12.464, 12.321, 12.178, 12.031,
         12.012],
        [14.046, 13.532, 12.594, 12.472, 12.042, 11.971, 11.491, 11.430, 11.399,
         11.374],
        [15.141, 13.798, 13.274, 13.216, 13.098, 13.055, 13.001, 12.656, 12.628,
         12.502],
        [13.540, 12.035, 11.279, 10.751,  9.803,  9.763,  9.549,  9.433,  9.3

In [11]:
masked_text

[' dirt. Unfortunately the majority of<mask> end up in<mask> that you had to<mask><mask><mask> My only recourse is to<mask> it myself. What would be the happy tale of my life then?',
 ' wearing games and<mask> ****ing<mask> do I hate horse wearing games.',
 " fetishes: it just makes me want to<mask><mask> every time I see it on the<mask><mask> even though it's not worth a thing because I am<mask><mask><mask>"]

In [13]:
## replace tokens at the indices with mask tokens

indices_in_mlm_tokens_old = []
predicted_token_ids_old = []
for masked_text_ in masked_text:
    
    ## replace tokens at the indices with mask tokens
    inputs_old = mlm_tokenizer(
        masked_text_, return_tensors="pt"
    )
    inputs_old = inputs_old.to(config['device'])
    ## make predictions for the masked indices
    with torch.no_grad():
        logits_old = mlm(**inputs_old).logits
    indices_in_mlm_tokens_old_ = (
        inputs_old.input_ids == mlm_tokenizer.mask_token_id
    )[0].nonzero(as_tuple=True)[0]
    # print(f"indices_in_mlm_tokens: {indices_in_mlm_tokens}")
    ## get top k tokens for each index
    
    ## make logits for special tokens -inf.
    special_token_ids = mlm_tokenizer.convert_tokens_to_ids(mlm_tokenizer.all_special_tokens)
    logits_old[:, :, special_token_ids] = -np.inf
    
    predicted_token_ids_old_ = torch.topk(
        logits_old[0, indices_in_mlm_tokens_old_],
        k=wandb.config.k_per_location,
        dim=-1,
    )
    indices_in_mlm_tokens_old.append(indices_in_mlm_tokens_old_)
    predicted_token_ids_old.append(predicted_token_ids_old_)

In [15]:
indices_in_mlm_tokens

(tensor([0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2], device='cuda:0'),
 tensor([ 7, 11, 16, 17, 18, 24,  4,  7, 10, 11, 19, 20, 32, 33, 34],
        device='cuda:0'))

In [14]:
indices_in_mlm_tokens_old

[tensor([ 7, 11, 16, 17, 18, 24], device='cuda:0'),
 tensor([4, 7], device='cuda:0'),
 tensor([10, 11, 19, 20, 32, 33, 34], device='cuda:0')]

In [20]:
torch.cat([x.indices for x in predicted_token_ids_old],dim=0)

tensor([[   82,    47,   106,   201,   390,  1652,  1159,  2799, 28263, 21304],
        [ 5458,   402,  1126,  4158,  1789,   982,  2127,  2878,  4215,   749],
        [  697,   582,   213,   173,  3116,  1032,   432,  2807,   120,   244],
        [   13,  2512,    19,    62,     7,   149,    66,    31,    11,  2185],
        [    4,   116,   328,  1174,   734,   479,  1555,  1666,    72,     6],
        [  109,   146,  1032,  4190,  3116,   652,  2073,   697,   907,  1166],
        [  596,  9069,  2143, 14223,  1840,   141, 15410,   117, 30722,    30],
        [  596,   203,     6,  7105, 26536,  9069,   141, 34912,  2143,  1840],
        [  907,    28,   120, 16531,   109, 17605,  3529,  3211, 10064,   657],
        [   24,   402,    55,    62,   357,  2185, 24217,     6,   106,    42],
        [ 2888,  2441,  1012,  2014,   340,  3742,  1400, 15263,  3748,  2384],
        [    6,   111,    93,   126,   734,  1555,  1174,   480,  2441,  1400],
        [   45,   416,    10,    98,    

In [19]:
predicted_token_ids.indices

tensor([[   82,    47,   106,   201,   390,  1652,  1159,  2799, 28263, 21304],
        [ 5458,   402,  1126,  4158,  1789,   982,  2127,  2878,  4215,   749],
        [  697,   582,   213,   173,  3116,  1032,   432,  2807,   120,   244],
        [   13,  2512,    19,    62,     7,   149,    66,    31,    11,  2185],
        [    4,   116,   328,  1174,   734,   479,  1555,  1666,    72,     6],
        [  109,   146,  1032,  4190,  3116,   652,  2073,   697,   907,  1166],
        [  596,  9069,  2143, 14223,  1840,   141, 15410,   117, 30722,    30],
        [  596,   203,     6,  7105, 26536,  9069,   141, 34912,  2143,  1840],
        [  907,    28,   120, 16531,   109, 17605,  3529,  3211, 10064,   657],
        [   24,   402,    55,    62,   357,  2185, 24217,     6,   106,    42],
        [ 2888,  2441,  1012,  2014,   340,  3742,  1400, 15263,  3748,  2384],
        [    6,   111,    93,   126,   734,  1555,  1174,   480,  2441,  1400],
        [   45,   416,    10,    98,    

In [21]:
torch.cat([x.indices for x in predicted_token_ids_old],dim=0) == predicted_token_ids.indices

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True,